<a href="https://colab.research.google.com/github/vikashkodati/mygig/blob/main/TwelveLabs_Quickstart_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1wq_1YvSl7jCh1P3KME96fBfJHzUSUwuo?usp=sharing"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in  Colab</a>
  </td>
</table>

##**Installation of TwelveLabs SDKs**

In [ ]:
!pip install -U -q twelvelabs

##**Configure your API key. Add API Key as a env variable.**
Signup for TwelveLabs and Get your API keys [here](https://playground.twelvelabs.xyz/dashboard/api-key). No credit card is required to use the Free plan. The Free plan includes indexing of 600 mins of videos, which is enough for a small project.

*To run the following cell in a*
* *Colab Notebook, your API key must be stored it in a Colab Secret named TL_API_KEY. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [this](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for an example.*
* *In other Python environments, You can also use Python variables using `os.environ()`*



In [ ]:
from google.colab import userdata
TL_API_KEY=userdata.get('TL_API_KEY')

## **Generate Embeddings**
Use the Embed API to create multimodal embeddings that are contextual vector representations for your videos and texts. Twelve Labs video embeddings capture all the subtle cues and interactions between different modalities, including the visual expressions, body language, spoken words, and the overall context of the video, encapsulating the essence of all these modalities and their interrelations over time.

To create video embeddings, you must first upload your videos, and the platform must finish processing them. Uploading and processing videos require some time. Consequently, creating embeddings is an asynchronous process comprised of three steps:

1. Upload and process a video: When you start uploading a video, the platform creates a video embedding task and returns its unique task identifier.

2. Monitor the status of your video embedding task: Use the unique identifier of your task to check its status periodically until it's completed.

3. Retrieve the embeddings: After the video embedding task is completed, retrieve the video embeddings by providing the task identifier.
Learn more in the [docs](https://docs.twelvelabs.io/docs/create-video-embeddings)


In [ ]:
from twelvelabs import TwelveLabs
from typing import List
from twelvelabs.models.embed import EmbeddingsTask, SegmentEmbedding

def print_segments(segments: List[SegmentEmbedding], max_elements: int = 5):
    for segment in segments:
        print(
            f"  embedding_scope={segment.embedding_scope} start_offset_sec={segment.start_offset_sec} end_offset_sec={segment.end_offset_sec}"
        )
        print(f"  embeddings: {segment.embeddings_float[:max_elements]}")

client = TwelveLabs(api_key=TL_API_KEY)
task = client.embed.task.create(
    engine_name="Marengo-retrieval-2.6",
    video_url="https://sample-videos.com/video321/mp4/720/big_buck_bunny_720p_2mb.mp4" # # Example: https://sample-videos.com/video321/mp4/720/big_buck_bunny_720p_2mb.mp4
)
print(
    f"Created task: id={task.id} engine_name={task.engine_name} status={task.status}"
)

def on_task_update(task: EmbeddingsTask):
    print(f"  Status={task.status}")

status = task.wait_for_done(
    sleep_interval=2,
    callback=on_task_update
)
print(f"Embedding done: {status}")

task = task.retrieve()
if task.video_embedding is not None and task.video_embedding.segments is not None:
    print_segments(task.video_embedding.segments)


Created task: id=6740eaac5d3d064fe3588bb3 engine_name=Marengo-retrieval-2.6 status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=ready
Embedding done: ready
  embedding_scope=clip start_offset_sec=0.0 end_offset_sec=6.0
  embeddings: [-0.060122214, 0.016685927, -0.008494042, 0.007121656, 0.0040775263]
  embedding_scope=clip start_offset_sec=6.0 end_offset_sec=12.0
  embeddings: [-0.05643653, 0.012836819

## **GET Embeddings from a video in your INDEX.**
Replace `index_id` and `video_id` in the following code with your index and video ids.


In [ ]:
from twelvelabs import TwelveLabs
from typing import List
from twelvelabs.models.embed import SegmentEmbedding

def print_segments(segments: List[SegmentEmbedding], max_elements: int = 5):
    for segment in segments:
        print(
            f"  embedding_scope={segment.embedding_scope} start_offset_sec={segment.start_offset_sec} end_offset_sec={segment.end_offset_sec}"
        )
        print(f"  embeddings: {segment.embeddings_float[:max_elements]}")

client = TwelveLabs(api_key=TL_API_KEY)
index_id = "6733ac47f1f0c94d455e61af" # Use your index_id
video_id = "6733c3481a5f2afde1139307" # Use your video_id

video = client.index.video.retrieve(
    index_id=index_id, id=video_id, embed=True)
if video.embedding:
    print(f"Engine_name={video.embedding.engine_name}")
    print("Embeddings:")
    print_segments(video.embedding.video_embedding.segments)

Engine_name=Marengo-retrieval-2.6
Embeddings:
  embedding_scope=clip start_offset_sec=0.0 end_offset_sec=8.833333
  embeddings: [-0.05401685, 0.028044553, -0.006683201, 0.052161995, 0.005205433]
  embedding_scope=clip start_offset_sec=8.866667 end_offset_sec=14.333333
  embeddings: [-0.064734034, 0.015777389, 0.016971672, 0.06758459, -0.0042496417]
  embedding_scope=clip start_offset_sec=14.366667 end_offset_sec=20.1
  embeddings: [-0.04318019, 0.016767595, 0.031129684, 0.061615035, -0.033064734]
  embedding_scope=clip start_offset_sec=20.133333 end_offset_sec=27.199999
  embeddings: [-0.048935793, 0.0015533045, 0.04537521, 0.042919084, -0.003435039]
  embedding_scope=clip start_offset_sec=27.233334 end_offset_sec=34.233334
  embeddings: [-0.042636164, 0.016505633, 0.021705803, 0.060528487, -0.026437962]
  embedding_scope=clip start_offset_sec=34.266666 end_offset_sec=41.333332
  embeddings: [-0.03781607, 0.0047186627, 0.030991001, 0.046645194, -0.016894642]
  embedding_scope=clip star

# **Text, Image and Audio Embeddings**

In [ ]:
from twelvelabs import TwelveLabs
from typing import List
from twelvelabs.models.embed import EmbeddingsTask, SegmentEmbedding

def print_segments(segments: List[SegmentEmbedding], max_elements: int = 5):
    for segment in segments:
        print(f"  embeddings: {segment.embeddings_float[:max_elements]}")

def create_text_embedding(
    twelvelabs_client: TwelveLabs,
    text: str = "Hello World!",
    engine_name: str = "Marengo-retrieval-2.6",
    verbose: bool = True
) -> dict:
    """
    Create a text embedding using Twelve Labs Embed API.

    Example:
        twelvelabs_client = TwelveLabs(api_key=TL_API_KEY)
        text_embedding = create_text_embedding(twelvelabs_client, "Your text here")
    """
    # Create embedding
    result = twelvelabs_client.embed.create(
        engine_name=engine_name,
        text=text
    )

    # Print information if verbose is True
    if verbose:
        print("Created a text embedding")
        print_segments(result.text_embedding.segments)

    return result

def create_audio_embedding(
    twelvelabs_client: TwelveLabs,
    audio_url: str = "https://codeskulptor-demos.commondatastorage.googleapis.com/descent/background%20music.mp3",
    engine_name: str = "Marengo-retrieval-2.6",
    verbose: bool = True
) -> dict:
    """
    Create an audio embedding using Twelve Labs Embed API.

    Example:
        twelvelabs_client = TwelveLabs(api_key=TL_API_KEY)
        audio_embedding = create_audio_embedding(twelvelabs_client, "path/to/audio.mp3")
    """
    # Create embedding
    result = twelvelabs_client.embed.create(
        engine_name=engine_name,
        audio_url=audio_url,
    )

    # Print information if verbose is True
    if verbose:
        print("Created an audio embedding")
        if result.audio_embedding.segments:
            print_segments(result.audio_embedding.segments)

    return result

def create_image_embedding(
    twelvelabs_client: TwelveLabs,
    image_url:  str = "https://gratisography.com/wp-content/uploads/2024/01/gratisography-cyber-kitty-1170x780.jpg",
    engine_name: str = "Marengo-retrieval-2.6",
    verbose: bool = True
) -> dict:
    """
    Create an image embedding using Twelve Labs Embed API.

    Example:
        twelvelabs_client = TwelveLabs(api_key=TL_API_KEY)
        image_embedding = create_image_embedding(twelvelabs_client, "path/to/image.jpg")
    """
    # Create embedding
    result = twelvelabs_client.embed.create(
        engine_name=engine_name,
        image_url=image_url
    )

    # Print information if verbose is True
    if verbose:
        print("Created an image embedding")
        if result.image_embedding.segments:
            print_segments(result.image_embedding.segments)

    return result

In [ ]:
from twelvelabs import TwelveLabs
from typing import List
from twelvelabs.models.embed import EmbeddingsTask, SegmentEmbedding

twelvelabs_client = TwelveLabs(api_key=TL_API_KEY)

aem = create_audio_embedding(twelvelabs_client)
im = create_image_embedding(twelvelabs_client)
tm = create_text_embedding(twelvelabs_client)

Created an audio embedding
  embeddings: [-0.07128955, 0.010958959, 0.01905033, -0.013187495, 0.068803824]
Created an image embedding
  embeddings: [-0.03812987, 0.013206651, 0.016828481, 0.06474987, 0.007665184]
Created a text embedding
  embeddings: [0.061489083, 0.020508073, -0.039171655, -0.004987218, 0.015630098]
